# MobileNetV2 transfer learning

### 1. Load data

In [1]:
import optuna
from keras import callbacks
import numpy as np

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os

def reduce_nondeterminism():
    import os
    os.environ['TF_CUDNN_DETERMINISTIC']='1'
    import tensorflow as tf
    import random as rn
    import numpy as np
    tf.random.set_seed(42)
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(42)
    rn.seed(42)
    
reduce_nondeterminism()

labels = ['PNEUMONIA', 'NORMAL']
img_size = 160
def get_training_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
                # print(resized_arr.shape)
            except Exception as e:
                print(e)
    return np.array(data, dtype=object)
    # return np.array(data)

train = get_training_data('chest_xray/chest_xray/train')
test = get_training_data('chest_xray/chest_xray/test')
val = get_training_data('chest_xray/chest_xray/val')

x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

# resize data for deep learning
x_train = x_train.reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 3)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)


2023-09-01 18:34:37.731728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Data augmentation

In [2]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range=0.2,  # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

### 3. Neural Network architecture

In [3]:
learning_rate = 0.0033949725096571503
dropout = 0.2067618721070267

input_shape = (img_size, img_size, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False,  # <== Important!!!!
                                                   weights='imagenet')  # From imageNet

base_model.trainable = False
inputs = tf.keras.Input(shape=input_shape)  # TODO: is it necessary?
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(dropout)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # TODO: sigmoid?
model = tf.keras.Model(inputs, outputs)

2023-09-01 18:35:17.853900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-01 18:35:18.173408: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-01 18:35:18.173568: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-01 18:35:18.174075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

### 4. Fit model

In [4]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=METRICS)

checkpoint_filepath_first = "article_filapath.h5"
checkpoint_cb_first = tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath_first,
                                                             save_best_only=True, monitor="val_loss", mode="min")
earlystopping = callbacks.EarlyStopping(monitor="val_loss",
                                            mode="min", patience=20,
                                            restore_best_weights=True)
batch_size = 32
epochs_first_fitting = 200
epochs_fine_tunning = 200
hist = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), batch_size=batch_size,
                 epochs=epochs_first_fitting,
                 verbose=2,
                 validation_data=datagen.flow(x_val, y_val),
                 callbacks=[checkpoint_cb_first, earlystopping])

Epoch 1/200


2023-09-01 18:35:23.167035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-09-01 18:35:24.790042: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-01 18:35:24.869127: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f47948cc6b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-01 18:35:24.869141: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2023-09-01 18:35:24.872150: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-01 18:35:24.913031: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-01 18:35:24.937628: I tensorflow/compiler/jit/xla_compil

139/139 - 18s - loss: 0.2437 - accuracy: 0.8982 - precision: 0.8194 - recall: 0.7525 - val_loss: 0.2035 - val_accuracy: 0.9297 - val_precision: 0.9694 - val_recall: 0.7422 - 18s/epoch - 130ms/step
Epoch 2/200
139/139 - 13s - loss: 0.1808 - accuracy: 0.9237 - precision: 0.8483 - recall: 0.8405 - val_loss: 0.1602 - val_accuracy: 0.9434 - val_precision: 0.9626 - val_recall: 0.8047 - 13s/epoch - 94ms/step
Epoch 3/200
139/139 - 13s - loss: 0.1692 - accuracy: 0.9307 - precision: 0.8603 - recall: 0.8579 - val_loss: 0.2055 - val_accuracy: 0.9180 - val_precision: 0.9574 - val_recall: 0.7031 - 13s/epoch - 93ms/step
Epoch 4/200
139/139 - 13s - loss: 0.1552 - accuracy: 0.9384 - precision: 0.8808 - recall: 0.8671 - val_loss: 0.1797 - val_accuracy: 0.9375 - val_precision: 0.9615 - val_recall: 0.7812 - 13s/epoch - 91ms/step
Epoch 5/200
139/139 - 13s - loss: 0.1581 - accuracy: 0.9364 - precision: 0.8714 - recall: 0.8698 - val_loss: 0.2124 - val_accuracy: 0.9258 - val_precision: 0.9592 - val_recall: 0.

Epoch 41/200
139/139 - 12s - loss: 0.1478 - accuracy: 0.9431 - precision: 0.8838 - recall: 0.8854 - val_loss: 0.2055 - val_accuracy: 0.9316 - val_precision: 0.9794 - val_recall: 0.7422 - 12s/epoch - 89ms/step
Epoch 42/200
139/139 - 13s - loss: 0.1494 - accuracy: 0.9404 - precision: 0.8797 - recall: 0.8781 - val_loss: 0.1827 - val_accuracy: 0.9316 - val_precision: 0.8720 - val_recall: 0.8516 - 13s/epoch - 91ms/step
Epoch 43/200
139/139 - 13s - loss: 0.1413 - accuracy: 0.9436 - precision: 0.8854 - recall: 0.8854 - val_loss: 0.1531 - val_accuracy: 0.9414 - val_precision: 0.9537 - val_recall: 0.8047 - 13s/epoch - 93ms/step


### 5. Evaluate the model

In [5]:
model.load_weights(checkpoint_filepath_first)
eval = model.evaluate(x_test, y_test)
print("Loss of the model is - ", eval[0])
print("Accuracy of the model is - ", eval[1] * 100, "%")

20/20 [==============================] - 0s 17ms/step - loss: 0.5031 - accuracy: 0.8077 - precision: 0.6629 - recall: 0.9915
Loss of the model is -  0.50307297706604
Accuracy of the model is -  80.7692289352417 %


### 6. Fine tune more layers

In [6]:
fine_tune_at = 119
model.load_weights(checkpoint_filepath_first)
base_model = model.layers[1]
base_model.trainable = True
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = fine_tune_at
loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1 * learning_rate)
model.compile(loss=loss_function,
                  optimizer=optimizer,
                  metrics=METRICS)
checkpoint_filepath = "fine_tune_final.h5"
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath,
                                                       save_best_only=True, monitor="val_loss", mode="min")
hist_fine = model.fit(datagen.flow(x_train, y_train, batch_size=32), batch_size=batch_size,
                          epochs=epochs_fine_tunning,
                          verbose=2,
                          validation_data=datagen.flow(x_val, y_val),
                          callbacks=[checkpoint_cb, earlystopping],
                          initial_epoch=hist.epoch[-1])


Epoch 43/200


2023-09-01 18:44:44.050023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


139/139 - 25s - loss: 0.8636 - accuracy: 0.7450 - precision: 0.5285 - recall: 0.5245 - val_loss: 0.5463 - val_accuracy: 0.7520 - val_precision: 0.5021 - val_recall: 0.9141 - 25s/epoch - 181ms/step
Epoch 44/200
139/139 - 13s - loss: 0.7132 - accuracy: 0.8228 - precision: 0.6406 - recall: 0.6389 - val_loss: 0.3947 - val_accuracy: 0.9238 - val_precision: 0.9083 - val_recall: 0.7734 - 13s/epoch - 97ms/step
Epoch 45/200
139/139 - 13s - loss: 0.5377 - accuracy: 0.8556 - precision: 0.7094 - recall: 0.7003 - val_loss: 0.3485 - val_accuracy: 0.9316 - val_precision: 0.9043 - val_recall: 0.8125 - 13s/epoch - 95ms/step
Epoch 46/200
139/139 - 13s - loss: 0.4152 - accuracy: 0.8937 - precision: 0.7865 - recall: 0.7800 - val_loss: 0.4703 - val_accuracy: 0.8848 - val_precision: 0.9859 - val_recall: 0.5469 - 13s/epoch - 92ms/step
Epoch 47/200
139/139 - 13s - loss: 0.3219 - accuracy: 0.9156 - precision: 0.8298 - recall: 0.8268 - val_loss: 0.4773 - val_accuracy: 0.9004 - val_precision: 0.9873 - val_recall

Epoch 82/200
139/139 - 13s - loss: 1.0236 - accuracy: 0.6385 - precision: 0.2214 - recall: 0.1861 - val_loss: 0.6311 - val_accuracy: 0.7500 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 13s/epoch - 93ms/step
Epoch 83/200
139/139 - 13s - loss: 0.9579 - accuracy: 0.6491 - precision: 0.2428 - recall: 0.2007 - val_loss: 0.6561 - val_accuracy: 0.7500 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 13s/epoch - 96ms/step
Epoch 84/200
139/139 - 13s - loss: 0.9106 - accuracy: 0.6628 - precision: 0.2723 - recall: 0.2209 - val_loss: 0.5908 - val_accuracy: 0.7500 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 13s/epoch - 92ms/step
Epoch 85/200
139/139 - 13s - loss: 0.8657 - accuracy: 0.6513 - precision: 0.2304 - recall: 0.1778 - val_loss: 0.6957 - val_accuracy: 0.7500 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 13s/epoch - 91ms/step


In [7]:
model.load_weights(checkpoint_filepath)
eval = model.evaluate(x_test, y_test)
print("Loss of the model is - ", eval[0])
print("Accuracy of the model is - ", eval[1] * 100, "%")

20/20 [==============================] - 0s 14ms/step - loss: 0.2110 - accuracy: 0.9471 - precision: 0.9763 - recall: 0.8803 
Loss of the model is -  0.2109663039445877
Accuracy of the model is -  94.71153616905212 %
